<a href="https://colab.research.google.com/github/Mhmdaris15/Covid-19-in-Indonesia/blob/main/COVID_19_Indonesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.preprocessing import add_dummy_feature, MinMaxScaler, OneHotEncoder, StandardScaler, RobustScaler, Normalizer

In [2]:
from google.colab import files
files.upload()

!pip install kaggle

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d hendratno/covid19-indonesia

!unzip /content/covid19-indonesia.zip

Saving kaggle.json to kaggle.json
  0% 0.00/489k [00:00<?, ?B/s]
100% 489k/489k [00:00<00:00, 31.4MB/s]
Archive:  /content/covid19-indonesia.zip
  inflating: covid_19_indonesia_time_series_all.csv  


In [3]:
df = pd.read_csv('covid_19_indonesia_time_series_all.csv')
df

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,Total Active Cases,Location Level,City or Regency,Province,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,489,35,39,415,Province,NaN,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,NaN,664,10846145,16334.31,106.836118,-6.204699,0.18,45.09,0.00,3.23,7.16%,7.98%,NaN,NaN
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,491,35,39,417,Province,NaN,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,NaN,664,10846145,16334.31,106.836118,-6.204699,0.18,45.27,0.00,3.23,7.13%,7.94%,1.00,1.00
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,2,Country,NaN,NaN,Indonesia,Asia,NaN,NaN,NaN,416,98.0,7230,8488.0,74953.0,1916907,265185520,138.34,113.921327,-0.789275,0.01,0.01,0.00,0.00,0.00%,0.00%,NaN,NaN
3,3/2/2020,ID-JB,Jawa Barat,3,0,0,3,12,10,624,-622,Province,NaN,Jawa Barat,Indonesia,Asia,Jawa,UTC+07:00,NaN,18,9.0,627,645.0,5312.0,35378,45161325,1276.55,107.603708,-6.920432,0.07,0.27,0.00,0.22,83.33%,5200.00%,NaN,NaN
4,3/2/2020,ID-RI,Riau,1,0,0,1,2,2,8,-8,Province,NaN,Riau,Indonesia,Asia,Sumatera,UTC+07:00,NaN,10,2.0,169,268.0,1591.0,87024,6074100,69.80,101.805109,0.511648,0.16,0.33,0.00,0.33,100.00%,400.00%,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13009,3/28/2021,ID-SG,Sulawesi Tenggara,4,0,1,3,10249,205,9593,451,Province,NaN,Sulawesi Tenggara,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,15,2.0,219,377.0,1911.0,38068,2635461,69.23,122.070311,-4.124689,1.52,3888.88,0.00,77.79,2.00%,93.60%,0.31,0.00
13010,3/28/2021,ID-SA,Sulawesi Utara,5,0,4,1,15351,499,12420,2432,Province,NaN,Sulawesi Utara,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,11,4.0,171,332.0,1507.0,13892,2641884,190.17,124.521240,1.259638,1.89,5810.63,0.00,188.88,3.25%,80.91%,0.26,1.00
13011,3/28/2021,ID-SB,Sumatera Barat,106,2,74,30,31455,682,29611,1162,Province,NaN,Sumatera Barat,Indonesia,Asia,Sumatera,UTC+07:00,NaN,12,7.0,179,230.0,928.0,42013,5519245,131.37,100.465062,-0.850253,19.21,5699.15,0.36,123.57,2.17%,94.14%,0.96,2.00
13012,3/28/2021,ID-SS,Sumatera Selatan,101,3,77,21,17443,828,15450,1165,Province,NaN,Sumatera Selatan,Indonesia,Asia,Sumatera,UTC+07:00,NaN,13,4.0,241,387.0,2853.0,91592,8217551,89.72,104.169465,-3.216212,12.29,2122.65,0.37,100.76,4.75%,88.57%,1.20,1.50


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13014 entries, 0 to 13013
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         13014 non-null  object 
 1   Location ISO Code            13014 non-null  object 
 2   Location                     13014 non-null  object 
 3   New Cases                    13014 non-null  int64  
 4   New Deaths                   13014 non-null  int64  
 5   New Recovered                13014 non-null  int64  
 6   New Active Cases             13014 non-null  int64  
 7   Total Cases                  13014 non-null  int64  
 8   Total Deaths                 13014 non-null  int64  
 9   Total Recovered              13014 non-null  int64  
 10  Total Active Cases           13014 non-null  int64  
 11  Location Level               13014 non-null  object 
 12  City or Regency              0 non-null      float64
 13  Province        

In [5]:
df.isna().sum()

Date                               0
Location ISO Code                  0
Location                           0
New Cases                          0
New Deaths                         0
New Recovered                      0
New Active Cases                   0
Total Cases                        0
Total Deaths                       0
Total Recovered                    0
Total Active Cases                 0
Location Level                     0
City or Regency                13014
Province                         392
Country                            0
Continent                          0
Island                           392
Time Zone                        392
Special Status                 11136
Total Regencies                    0
Total Cities                     366
Total Districts                    0
Total Urban Villages             368
Total Rural Villages             393
Area (km2)                         0
Population                         0
Population Density                 0
L

---
Dari dataset tersebut banyak NaN Value yang muncul, yaitu pada column : <br>
* `City or Agency`
* `Province`
* `Island`
* `Time Zone`
* `Special Status`
* `Total Cities`
* `Total Urban Villages`
* `Total Rural Villages`
* `Growth Factor of New Cases`
* `Growth Factor of New Deaths`



In [6]:
print(df[df['Total Cities'].isna()].Location.unique())
print(df[df['Island'].isna()].Location.unique())
print(df[df['Time Zone'].isna()].Location.unique())
print(df[df['Total Urban Villages'].isna()].Location.unique())
print(df[df['Total Rural Villages'].isna()].Location.unique())


['Sulawesi Barat']
['Indonesia']
['Indonesia']
['Aceh']
['DKI Jakarta']


In [20]:
df_cleaned = df.drop(columns=['City or Regency'])
df_cleaned['Special Status'].fillna('General', inplace=True)
df_cleaned['Island'].fillna('Unknown', inplace=True)
df_cleaned['Province'].fillna('Unknown', inplace=True)
df_cleaned['Time Zone'].fillna('Indonesia Time Zone', inplace=True)
df_cleaned['Total Cities'].fillna(0, inplace=True)
df_cleaned['Total Urban Villages'].fillna(0, inplace=True)
df_cleaned['Total Rural Villages'].fillna(0, inplace=True)

In [22]:
df_cleaned.isna().sum()

Date                              0
Location ISO Code                 0
Location                          0
New Cases                         0
New Deaths                        0
New Recovered                     0
New Active Cases                  0
Total Cases                       0
Total Deaths                      0
Total Recovered                   0
Total Active Cases                0
Location Level                    0
Province                          0
Country                           0
Continent                         0
Island                            0
Time Zone                         0
Special Status                    0
Total Regencies                   0
Total Cities                      0
Total Districts                   0
Total Urban Villages              0
Total Rural Villages              0
Area (km2)                        0
Population                        0
Population Density                0
Longitude                         0
Latitude                    

In [42]:
df_cleaned[df_cleaned['Growth Factor of New Cases'].isna()].Location.nunique()

35

In [45]:
df_cleaned.Location.nunique()

35

In [63]:
nan_idx_last = np.where(df['Growth Factor of New Cases'].isna())[0][:5]
df_cleaned.iloc[nan_idx_last,:]

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,Total Active Cases,Location Level,Province,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,489,35,39,415,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,0.18,45.09,0.0,3.23,7.16%,7.98%,NaN,NaN
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,2,Country,Unknown,Indonesia,Asia,Unknown,Indonesia Time Zone,General,416,98.0,7230,8488.0,74953.0,1916907,265185520,138.34,113.921327,-0.789275,0.01,0.01,0.0,0.00,0.00%,0.00%,NaN,NaN
3,3/2/2020,ID-JB,Jawa Barat,3,0,0,3,12,10,624,-622,Province,Jawa Barat,Indonesia,Asia,Jawa,UTC+07:00,General,18,9.0,627,645.0,5312.0,35378,45161325,1276.55,107.603708,-6.920432,0.07,0.27,0.0,0.22,83.33%,5200.00%,NaN,NaN
4,3/2/2020,ID-RI,Riau,1,0,0,1,2,2,8,-8,Province,Riau,Indonesia,Asia,Sumatera,UTC+07:00,General,10,2.0,169,268.0,1591.0,87024,6074100,69.80,101.805109,0.511648,0.16,0.33,0.0,0.33,100.00%,400.00%,NaN,NaN
17,3/6/2020,ID-BT,Banten,1,0,0,1,59,1,31,27,Province,Banten,Indonesia,Asia,Jawa,UTC+07:00,General,4,4.0,155,313.0,1238.0,9663,10722374,1109.64,106.109004,-6.456736,0.09,5.50,0.0,0.09,1.69%,52.54%,NaN,NaN


In [69]:
print(df_cleaned[df['Location'] == 'DKI Jakarta'].iloc[:,-2].mean())
print(df_cleaned[df['Location'] == 'Riau'].iloc[:,-2].mean())
print(df_cleaned[df['Location'] == 'Banten'].iloc[:,-2].mean())

1.0639432989690722
1.153497267759562
1.6521866666666656


In [81]:
avg1 = df_cleaned.loc[df_cleaned.Location == 'DKI Jakarta', 'Growth Factor of New Cases'].mean()
df_cleaned.loc[df_cleaned.Location == 'DKI Jakarta','Growth Factor of New Cases'].fillna(avg1, inplace=True)

In [82]:
df_cleaned[df_cleaned.Location == 'DKI Jakarta']

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,Total Active Cases,Location Level,Province,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,489,35,39,415,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,0.18,45.09,0.00,3.23,7.16%,7.98%,NaN,NaN
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,491,35,39,417,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,0.18,45.27,0.00,3.23,7.13%,7.94%,1.00,1.00
5,3/3/2020,ID-JK,DKI Jakarta,2,0,0,2,493,35,39,419,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,0.18,45.45,0.00,3.23,7.10%,7.91%,1.00,1.00
9,3/4/2020,ID-JK,DKI Jakarta,2,0,0,2,495,35,39,421,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,0.18,45.64,0.00,3.23,7.07%,7.88%,1.00,1.00
13,3/5/2020,ID-JK,DKI Jakarta,0,1,0,-1,495,36,39,420,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,0.00,45.64,0.09,3.32,7.27%,7.88%,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12843,3/24/2021,ID-JK,DKI Jakarta,890,11,1987,-1108,373759,6252,362439,5068,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,82.06,34460.08,1.01,576.43,1.67%,96.97%,1.09,1.00
12878,3/25/2021,ID-JK,DKI Jakarta,1727,12,490,1225,375486,6264,362929,6293,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,159.23,34619.30,1.11,577.53,1.67%,96.66%,1.94,1.09
12913,3/26/2021,ID-JK,DKI Jakarta,1381,11,897,473,376867,6275,363826,6766,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,127.33,34746.63,1.01,578.55,1.67%,96.54%,0.80,0.92
12948,3/27/2021,ID-JK,DKI Jakarta,1354,18,599,737,378221,6293,364425,7503,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,124.84,34871.47,1.66,580.21,1.66%,96.35%,0.98,1.64


In [70]:
for locate in df_cleaned.Location:
  avg1 = df_cleaned.loc[df_cleaned.Location == locate, 'Growth Factor of New Cases'].mean()
  avg2 = df_cleaned.loc[df_cleaned.Location == locate, 'Growth Factor of New Deaths'].mean()
  df_cleaned.loc[df_cleaned.Location == locate,'Growth Factor of New Cases'].fillna(avg1, inplace=True)
  df_cleaned.loc[df_cleaned.Location == locate,'Growth Factor of New Deaths'].fillna(avg2, inplace=True)

In [78]:
df_cleaned.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,Total Active Cases,Location Level,Province,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,489,35,39,415,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,0.18,45.09,0.0,3.23,7.16%,7.98%,NaN,NaN
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,491,35,39,417,Province,DKI Jakarta,Indonesia,Asia,Jawa,UTC+07:00,Daerah Khusus Ibu Kota,1,5.0,44,267.0,0.0,664,10846145,16334.31,106.836118,-6.204699,0.18,45.27,0.0,3.23,7.13%,7.94%,1.0,1.0
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,2,Country,Unknown,Indonesia,Asia,Unknown,Indonesia Time Zone,General,416,98.0,7230,8488.0,74953.0,1916907,265185520,138.34,113.921327,-0.789275,0.01,0.01,0.0,0.00,0.00%,0.00%,NaN,NaN
3,3/2/2020,ID-JB,Jawa Barat,3,0,0,3,12,10,624,-622,Province,Jawa Barat,Indonesia,Asia,Jawa,UTC+07:00,General,18,9.0,627,645.0,5312.0,35378,45161325,1276.55,107.603708,-6.920432,0.07,0.27,0.0,0.22,83.33%,5200.00%,NaN,NaN
4,3/2/2020,ID-RI,Riau,1,0,0,1,2,2,8,-8,Province,Riau,Indonesia,Asia,Sumatera,UTC+07:00,General,10,2.0,169,268.0,1591.0,87024,6074100,69.80,101.805109,0.511648,0.16,0.33,0.0,0.33,100.00%,400.00%,NaN,NaN


[link text](https://)